<a href="https://colab.research.google.com/github/alangrubb99/deep-learning-challenge/blob/main/Alphabet_Soup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [ ]:
import os
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,430 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,471 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/univers

In [ ]:
#Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

In [ ]:
# Start a SparkSession
import findspark
findspark.init()


In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from pyspark.sql import SparkSession
import time



#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)
application_df



,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
apps = application_df['APPLICATION_TYPE'].value_counts(),


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#df1['binned'] = pd.cut(df1['Score'], bins,labels=labels)
#print (df1)
application_types_to_replace = ['T12','T2','T25','T14','T29','T15','T17','T13']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#application_df['CLASSIFICATION'].value_counts().gt(1).sum()
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x<70]


C1300    58
C1280    50
C1230    36
C1400    34
C7200    32
C2300    32
C1240    30
C8000    20
C7120    18
C1500    16
C1800    15
C6000    15
C1250    14
C8200    11
C1238    10
C1278    10
C1235     9
C1237     9
C7210     7
C2400     6
C1720     6
C4100     6
C1257     5
C1600     5
C1260     3
C2710     3
C0        3
C3200     2
C1234     2
C1246     2
C1267     2
C1256     2
C2190     1
C4200     1
C2600     1
C5200     1
C1370     1
C1248     1
C6100     1
C1820     1
C1900     1
C1236     1
C3700     1
C2570     1
C1580     1
C1245     1
C2500     1
C1570     1
C1283     1
C2380     1
C1732     1
C1728     1
C2170     1
C4120     1
C8210     1
C2561     1
C4500     1
C2150     1
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace'
classifications_to_replace = ['C4120','C8210','C2561','C4500','C2150','C1235',
                              'C1237','C7120','C1500','C1800','C6000','C1250','C8200','C1238','C1278',
                                'C7210','C1230','C1400','C7200','C2300','C1240','C8000',
                                'C2400','C1300','C1280',
                                'C1720',
                                'C4100',
                                'C1257',
                                'C1600',
                                'C1260',
                                'C2710',
                                'C0',
'C3200',
'C1234',
'C1246',
'C1267',
'C1256',
'C2190',
'C4200',
'C2600',
'C5200',
'C1370',
'C1248',
'C6100',
'C1820',
'C1900',
'C1236',
'C3700',
'C2570',
'C1580',
'C1245',
'C2500',
'C1570',
'C1283',
'C2380',
'C1732',
'C1728',
'C2170',
'C4120',
'C8210',
'C2561',
'C4500',
'C2150']

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      499
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns='IS_SUCCESSFUL')
X = pd.get_dummies(X)
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [ ]:
# Split our preprocessed data into our features and target arrays
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE

(25724, 52)

In [ ]:
X_test.shape


(8575, 52)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled

array([[ 0.01079982, -0.02957055, -0.06120378, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.02957055, -0.06120378, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.02957055, -0.06120378, ..., -0.07529085,
         0.02925687, -0.02925687],
       ...,
       [ 0.01079982, -0.02957055, -0.06120378, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.02957055, -0.06120378, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.02957055, -0.06120378, ..., -0.07529085,
         0.02925687, -0.02925687]])

In [ ]:
X_test_scaled

array([[ 0.01079982, -0.02957055, -0.06120378, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.02957055, -0.06120378, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.0293783 , -0.06120378, ..., -0.07529085,
         0.02925687, -0.02925687],
       ...,
       [ 0.01079982, -0.02957055, -0.06120378, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.02957055, -0.06120378, ..., -0.07529085,
         0.02925687, -0.02925687],
       [ 0.01079982, -0.02957055, -0.06120378, ..., -0.07529085,
         0.02925687, -0.02925687]])

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=2, activation="relu", input_dim=52))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=3, activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=4, activation="relu"))
# Output layer

nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 106       
                                                                 
 dense_1 (Dense)             (None, 3)                 9         
                                                                 
 dense_2 (Dense)             (None, 4)                 16        
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 136 (544.00 Byte)
Trainable params: 136 (544.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 4s 4ms/step - loss: 0.6417 - accuracy: 0.6569
Epoch 2/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5909 - accuracy: 0.7153
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5799 - accuracy: 0.7215
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5783 - accuracy: 0.7226
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5759 - accuracy: 0.7231
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5750 - accuracy: 0.7233
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5748 - accuracy: 0.7229
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5748 - accuracy: 0.7244
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5743 - accuracy: 0.7233
Epoch 10/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5737 - accuracy: 0.7252

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5657 - accuracy: 0.7249 - 503ms/epoch - 2ms/step
Loss: 0.5657138228416443, Accuracy: 0.7248979806900024


In [ ]:
# Export our model to HDF5 file
print("The model is saved to HDF5 format")
nn_model.save('/content/drive/MyDrive/AlphabetSoupCharity.h5')

The model is saved to HDF5 format


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
